In [4]:
## Some basic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
## Loading the datasets

dallas = pd.read_json('../Demographic/Dallas.json/Dallas.json')
losAngeles = pd.read_json('../Demographic/Los Angeles.json/Los Angeles.json')
philadelphia = pd.read_json('../Demographic/Philadelphia.json/Philadelphia.json')

jsonDatasets = [dallas, losAngeles, philadelphia]

parcelDallas = pd.read_csv('../parcelData/cleanDallas.csv')
parcelPhiladelphia = pd.read_csv('../parcelData/cleanPhiladelphia.csv')
parcelSocal = pd.read_csv('../parcelData/cleanSocal.csv')

parcelDallas['City'] = 'Dallas'
parcelPhiladelphia['City'] = 'Philadelpia'
parcelSocal['City'] = 'Socal'

parcelDallas['recrdareano'] = 0
parcelDallas['saleprice'] = 0
parcelDallas['numstories'] = 0
parcelDallas['taxamt'] = 0

parcelSocal['numstories'] = 0
parcelSocal['taxamt'] = 0

parcelCombined = pd.concat([parcelDallas, parcelPhiladelphia, parcelSocal])

print(parcelCombined)

                             owner            mailadd  ll_gissqft  ll_gisacre  \
0          HARDY SAMANTHA GILMORE        110 E 7TH ST       15916     0.36537   
1                   HUFFMAN JOHN E      321 S MAIN ST       12762     0.29297   
2     HUDSPETH STEVEN WAYNE EST OF      2505 W 6TH ST        8085     0.18559   
3     GONZALEZRIVERA SANTOS MARTIN  1205 ARROWHEAD DR        7220     0.16575   
4                  GOMEZ FREDDY J        1438 ELMS RD        9004     0.20670   
..                             ...                ...         ...         ...   
259              TA PROPERTIES LTD   1537 PONTIUS AVE       58397     1.34057   
260         LOCKNESS ZONE 1606 LLC                NaN       18682     0.42886   
261                COLUMBIA RE LLC                NaN       23281     0.53445   
262                       RIBITSCH  1405 VIA CATALUNA        8374     0.19224   
263  COUNTRY HILLS INVESTMENTS LLC                NaN       18083     0.41512   

     landval   parval      

In [14]:
parcelCombined = parcelCombined.drop(['owner', 'owner2', 'owner3', 'mailadd', 'parcelnumb', 'qoz', 'saledate', 'zoning_description', 'structno', 'usedesc'], axis=1)
print(parcelCombined)

     ll_gissqft  ll_gisacre  landval   parval zoning    City  recrdareano  \
0         15916     0.36537    40000   193080      1  Dallas            0   
1         12762     0.29297    63790   276580      1  Dallas            0   
2          8085     0.18559    35000   147410      1  Dallas            0   
3          7220     0.16575    25000   103660      1  Dallas            0   
4          9004     0.20670    22500   130710      1  Dallas            0   
..          ...         ...      ...      ...    ...     ...          ...   
259       58397     1.34057   360519  1162088   LAM2   Socal        42235   
260       18682     0.42886  1664640  2809080   LAM2   Socal         9638   
261       23281     0.53445  1414008  2676515   LAM2   Socal        11002   
262        8374     0.19224    51494    82883  LARD2   Socal         1960   
263       18083     0.41512  1388758  2083135   LAM2   Socal         8620   

     saleprice  numstories  taxamt  
0          0.0           0       0  
1

In [15]:
from sklearn.preprocessing import OneHotEncoder

parcelCombined = pd.get_dummies(parcelCombined, columns=['zoning'], prefix='zone')
parcelCombined = pd.get_dummies(parcelCombined, columns=['City'], prefix='zone')
print(parcelCombined)

     ll_gissqft  ll_gisacre  landval   parval  recrdareano  saleprice  \
0         15916     0.36537    40000   193080            0        0.0   
1         12762     0.29297    63790   276580            0        0.0   
2          8085     0.18559    35000   147410            0        0.0   
3          7220     0.16575    25000   103660            0        0.0   
4          9004     0.20670    22500   130710            0        0.0   
..          ...         ...      ...      ...          ...        ...   
259       58397     1.34057   360519  1162088        42235        0.0   
260       18682     0.42886  1664640  2809080         9638  2700027.0   
261       23281     0.53445  1414008  2676515        11002  2120021.0   
262        8374     0.19224    51494    82883         1960       -1.0   
263       18083     0.41512  1388758  2083135         8620  1650016.0   

     numstories  taxamt  zone_0  zone_1  ...  zone_TOMPBLC  zone_TOMR-LM  \
0             0       0       0       1  ...   

In [16]:
X = parcelCombined.loc[:, parcelCombined.columns != 'landval']
y = parcelCombined['landval']

In [17]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.8, random_state=1)

In [18]:
# Start putting together the neural network 
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import Input

In [19]:
# Declare the classifier
classifier = Sequential()

In [20]:
# TODO - Add the incoming shape
classifier.add(Input(shape=21))
classifier.add(Dense(units=11, activation='relu'))
classifier.add(Dense(units=11, activation='relu'))
classifier.add(Dense(units=1))

classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                242       
                                                                 
 dense_1 (Dense)             (None, 11)                132       
                                                                 
 dense_2 (Dense)             (None, 1)                 12        
                                                                 
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
# Train the model
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

TypeError: Model.fit() got an unexpected keyword argument 'nb_epoch'

In [ ]:
result = classifier.predict(X_test)

In [ ]:
# Split into training sets 

X_train, X_test, Y_train, Y_test = pd.DataFrame()
for dataset in jsonDatasets:
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = getTrainSplit(dataset)

    X_train = pd.concat([X_train, temp_X_train], ignore_index=True)
    X_test = pd.concat([X_test, temp_X_test], ignore_index=True)
    Y_train = pd.concat([Y_train, temp_Y_train], ignore_index=True)
    Y_test = pd.concat([Y_test, temp_Y_test], ignore_index=True)

